# Generating Animations
--- 
Animation of two gaussians in 2D with one depicting the background and the other one being the signal. <br>
The probability to discard the background hypothesis is dependent on the position of the signal peak in reference to the background.<br>
One animation shows the case that the signals mean is varied in a grid wise procedure while the background is kept at the same place.<br>
The other animation shows the signal rotating around the background. In the regions where the network was no trained explicitly the values for discarding the background hypothesis cannot be seen as reliable.

### Import Modules

In [2]:
# extern modules
import torch
import numpy as np
import matplotlib.pyplot as plt
import imageio.v3 as iio

# intern modules
from PNN_model_2d import ParameterizedNeuralNet

## 1) Load the Model

In [3]:
# path to file
PATH: str = "../trained_models/PNN_2d.pth"

# load model from 
model = ParameterizedNeuralNet()
PNN_state_dict = torch.load(PATH)
model.load_state_dict(PNN_state_dict)

<All keys matched successfully>

## 2) Animation

### Generate Multiple Frames and Save in Dedicated Folder

In [4]:
# generating the coordinate system
x_data = np.linspace(-2, 2, 100)
y_data = np.linspace(-2, 2, 100)
coordinates = np.array([[x, y] for x in x_data for y in y_data])

# choosing the background to be at [0.5,0.5]
bg_mean = [0.5, 0.5]
background = np.multiply(np.ones_like(coordinates), np.array(bg_mean))
data = np.zeros((100**2, 6))

# covariance
cov = np.eye(2) * 0.02

# sizes
sizes = 1000

### Generate Grid Animation 

In [5]:
# range for scanning through NN
animation_range = np.linspace(0, 1, 10)

# generating the grid for the scanned region
signals = np.array([[x, y] for x in animation_range for y in animation_range])

# iterating through the NN
for signal_idx, signal in enumerate(signals):

    # creation data vector for the PNN
    data[:, 0:2] = coordinates
    data[:, 2:4] = background
    data[:, 4:6] = np.multiply(np.ones_like(coordinates), np.array(signal))

    # model evaluation
    res = model(torch.Tensor(data)).detach().numpy()

    # plot the heatmap for the whole coordinate grid
    plt.scatter(coordinates[:,0],coordinates[:,1], marker = '.', facecolor = 'none', edgecolor = plt.get_cmap('viridis')(res))

    # generate examples of the hypothesis
    example = [np.random.multivariate_normal(bg_mean, cov, size=sizes), np.random.multivariate_normal(signal, cov, size=sizes)]

    # plot examples
    plt.scatter(example[0][:,0], example[0][:, 1], marker=".", alpha=0.3)
    plt.scatter(example[1][:,0], example[1][:, 1], marker=".", alpha=0.3)
    plt.xlabel("x")
    plt.ylabel("y")

    # save the plot
    plt.savefig((f'../frames/2d_signal_grid_idx_{signal_idx}.png'))

    # flush the figure
    plt.close()

### Stack Multiple Frames in GIF

In [6]:
frames = np.stack([iio.imread(f'../frames/2d_signal_grid_idx_{signal_idx}.png') for signal_idx, signal in enumerate(signals)], axis=0)
iio.imwrite('../animations/2d_signal_grid.gif', frames)

### Display GIF

![](../animations/2d_signal_grid.gif)

### Generate Circular Animation

In [7]:
# angle for scanning through NN
phi = np.linspace(0, 2 * np.pi, 100)

# generating the grid for the scanned region
signals = np.array([[0.5 * np.cos(x) + 0.5, 0.5 * np.sin(x)+ 0.5] for x in phi])

# iterating through the NN
for signal_idx, signal in enumerate(signals):

    # creation data vector for the PNN
    data[:, 0:2] = coordinates
    data[:, 2:4] = background
    data[:, 4:6] = np.multiply(np.ones_like(coordinates), np.array(signal))

    # model evaluation
    res = model(torch.Tensor(data)).detach().numpy()

    # plot the heatmap for the whole coordinate grid
    plt.scatter(coordinates[:,0],coordinates[:,1], marker = '.', facecolor = 'none', edgecolor = plt.get_cmap('viridis')(res))

    # generate examples of the hypothesis
    example = [np.random.multivariate_normal(bg_mean, cov, size=sizes), np.random.multivariate_normal(signal, cov, size=sizes)]

    # plot examples
    plt.scatter(example[0][:,0], example[0][:, 1], marker=".", alpha=0.3)
    plt.scatter(example[1][:,0], example[1][:, 1], marker=".", alpha=0.3)
    plt.xlabel("x")
    plt.ylabel("y")

    # save the plot
    plt.savefig((f'../frames/2d_signal_circular_idx_{signal_idx}.png'))

    # flush the figure
    plt.close()
    

### Stack Multiple Frames in GIF

In [8]:
frames = np.stack([iio.imread(f'../frames/2d_signal_circular_idx_{signal_idx}.png') for signal_idx, signal in enumerate(signals)], axis=0)
iio.imwrite('../animations/2d_signal_circular.gif', frames)

### Display GIF

![](../animations/2d_signal_circular.gif)